# Cuaderno de Segmentación y Análisis Calibrado de Astrocitos

**Objetivo:** Este notebook detalla un flujo de trabajo para identificar núcleos de astrocitos, guardando y visualizando cada paso de filtrado. El análisis y la visualización están **calibrados con las dimensiones físicas del vóxel** para asegurar la precisión y reproducibilidad científica.

**Flujo de Trabajo:**
1.  **Configuración**: Definición de rutas y parámetros de calibración física y procesamiento.
2.  **Carga de Datos**: Carga de la imagen original.
3.  **Pre-procesamiento (Otsu)**: Limpieza del canal DAPI y guardado de la máscara binaria.
4.  **Segmentación (Cellpose)**: Segmentación de todos los núcleos y guardado de la máscara de Cellpose.
5.  **Filtrado (Co-localización GFAP)**: Selección de núcleos por señal GFAP y guardado de la máscara de candidatos.
6.  **Post-procesamiento (Tamaño Físico)**: Limpieza final por volumen físico (µm³) y guardado de la máscara final.

## Paso 0: Activar el Backend Gráfico

**Importante:** Ejecuta esta celda una sola vez por sesión. El comando mágico `%gui qt` prepara el notebook para mostrar ventanas interactivas como las de Napari.

In [8]:
%gui qt

## Paso 1: Configuración de Rutas y Parámetros

Esta celda centraliza todas las variables que controlan el flujo de trabajo. Definir los parámetros aquí permite ajustar fácilmente el análisis para diferentes imágenes sin modificar el código en las celdas posteriores.

### Fundamento del Filtrado y Reproducibilidad Científica 🔬

La estrategia de este cuaderno se basa en un **filtrado secuencial**, un método robusto y común en el análisis de imágenes biológicas. En lugar de depender de un único algoritmo "mágico", aplicamos una serie de filtros lógicos, cada uno diseñado para eliminar un tipo específico de artefacto o señal no deseada.

Este enfoque mejora la **reproducibilidad científica** por varias razones:
1.  **Transparencia**: Cada paso del filtrado (Otsu, Cellpose, Co-localización, Tamaño) es explícito y sus parámetros están claramente definidos.
2.  **Objetividad**: Al definir umbrales y parámetros numéricos, reducimos la subjetividad inherente a la selección manual.
3.  **Adaptabilidad**: Si se utiliza un nuevo set de imágenes, solo es necesario reajustar los parámetros en esta celda para adaptar el mismo flujo de trabajo lógico, manteniendo la consistencia del método.
4.  **Calibración**: El uso de las dimensiones físicas del vóxel permite que los umbrales se definan en unidades reales (µm³), haciendo que los resultados sean comparables entre diferentes microscopios o experimentos.

---

### Variables de Rutas de Archivos

* `base_filename` y `subfolder`: Permiten seleccionar fácilmente la imagen a procesar. El script construye las rutas de entrada y salida a partir de estos nombres, organizando los resultados en un directorio específico dentro de `/data/processed` que coincide con el nombre del archivo original.

---

### Variables de Calibración Física

* **`PIXEL_WIDTH_UM`, `PIXEL_HEIGHT_UM`, `Z_SPACING_UM`**:
    * **Uso**: En todo el cuaderno para calibrar las mediciones y visualizaciones.
    * **Explicación**: Definen las dimensiones físicas (en micrómetros) de un solo vóxel. Estos valores se extraen de los metadatos del archivo `.lif` original y son cruciales para convertir las mediciones de "conteo de vóxeles" en unidades físicas reales. La variable `PHYSICAL_SCALE` se pasa a Napari para asegurar que la visualización 3D tenga las proporciones correctas.

---

### Variables de Parámetros de Procesamiento

* **`NUCLEUS_DIAMETER`**:
    * **Uso**: En la **Celda de Segmentación con Cellpose**.
    * **Explicación**: Es el parámetro más importante para Cellpose. Le informa al modelo sobre el tamaño esperado (en píxeles) de los objetos que debe buscar. Un valor correcto mejora drásticamente la precisión de la segmentación. Se debe estimar midiendo el diámetro promedio de varios núcleos en la imagen original usando Napari.

* **`DILATION_ITERATIONS`**:
    * **Uso**: En la **Celda de Filtrado por Co-localización GFAP**.
    * **Explicación**: Controla el grosor del "anillo" que se crea alrededor de cada núcleo para medir la señal GFAP. Cada "iteración" expande la máscara del núcleo en un píxel en todas las direcciones. Un valor más alto crea un anillo más grueso y ligeramente más alejado.

* **`GFAP_INTENSITY_THRESHOLD`**:
    * **Uso**: En la **Celda de Filtrado por Co-localización GFAP**.
    * **Explicación**: Es el umbral de decisión para clasificar un núcleo como astrocito. El script calcula la intensidad promedio de la señal GFAP dentro del anillo peri-nuclear; si este promedio es mayor que el `GFAP_INTENSITY_THRESHOLD`, el núcleo es aceptado.

* **`MIN_VOLUME_UM3`**:
    * **Uso**: En la **Celda de Post-procesamiento por Tamaño**.
    * **Explicación**: Es el parámetro para el filtro de limpieza final, **definido en unidades físicas (micrómetros cúbicos, µm³)**. El script lo convierte automáticamente a un umbral en vóxeles (`MIN_VOLUME_VOXELS`) usando las dimensiones de calibración. Esto hace que el filtrado sea robusto y reproducible, independientemente de la resolución de la imagen.

In [5]:
from pathlib import Path

# --- Rutas de Archivos ---
project_root = Path.cwd().parent
base_filename = "Inmuno 26-07-23.lif - CTL 1-2 a"
subfolder = "CTL/CTL 1-2"
image_path = project_root / f"data/raw/{subfolder}/{base_filename}.tif"

# --- Rutas de Salida ---
output_dir = project_root / "data/processed" / base_filename
output_dir.mkdir(parents=True, exist_ok=True)
otsu_mask_path = output_dir / "01_otsu_mask.tif"
cellpose_mask_path = output_dir / "02_cellpose_mask.tif"
gfap_filtered_mask_path = output_dir / "03_gfap_filtered_mask.tif"
final_mask_path = output_dir / "04_final_astrocytes_mask.tif"

# --- PARÁMETROS DE CALIBRACIÓN FÍSICA (µm)---
PIXEL_WIDTH_UM = 0.3788
PIXEL_HEIGHT_UM = 0.3788
Z_SPACING_UM = 1.0071
VOXEL_VOLUME_UM3 = PIXEL_WIDTH_UM * PIXEL_HEIGHT_UM * Z_SPACING_UM
# Escala para Napari (debe coincidir con el orden de ejes Z, Y, X)
PHYSICAL_SCALE = (Z_SPACING_UM, PIXEL_HEIGHT_UM, PIXEL_WIDTH_UM)

# --- Parámetros de Procesamiento ---
NUCLEUS_DIAMETER = 30
DILATION_ITERATIONS = 40
GFAP_INTENSITY_THRESHOLD = 300
MIN_VOLUME_UM3 = 75 # Umbral de volumen en micrómetros cúbicos

# Convertimos el umbral físico a un umbral en vóxeles para el script
MIN_VOLUME_VOXELS = int(MIN_VOLUME_UM3 / VOXEL_VOLUME_UM3)
print(f"Calibración completa. Umbral de {MIN_VOLUME_UM3} µm³ equivale a {MIN_VOLUME_VOXELS} vóxeles.")

Calibración completa. Umbral de 75 µm³ equivale a 519 vóxeles.


## Paso 2: Carga de Datos e Importaciones

In [3]:
import tifffile
import numpy as np
import pandas as pd
from cellpose import models
from skimage.measure import regionprops
from skimage.filters import threshold_otsu
from scipy.ndimage import binary_dilation
import napari

print(f"Cargando imagen: {image_path.name}...")
image_stack = tifffile.imread(image_path)
dapi_channel = image_stack[:, 0, :, :]
gfap_channel = image_stack[:, 1, :, :]
print("Canales cargados.")



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	linux 
python version: 	3.12.3 
torch version:  	2.8.0+cu128! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 


Cargando imagen: Inmuno 26-07-23.lif - CTL 1-2 a.tif...
Canales cargados.


## Paso 3: Pre-procesamiento con Umbral de Otsu
Limpiamos el canal DAPI para eliminar ruido de fondo y guardamos el resultado.

In [6]:
print("Aplicando umbral de Otsu...")
otsu_threshold = threshold_otsu(dapi_channel)
otsu_mask = dapi_channel > otsu_threshold
dapi_channel_cleaned = np.where(otsu_mask, dapi_channel, 0)

tifffile.imwrite(otsu_mask_path, otsu_mask.astype(np.uint8))
print(f"Máscara de Otsu guardada en: {otsu_mask_path}")

Aplicando umbral de Otsu...
Máscara de Otsu guardada en: /home/daniel/Proyectos/astrocitos-3d-analysis/data/processed/Inmuno 26-07-23.lif - CTL 1-2 a/01_otsu_mask.tif


### Visualización: Máscara de Otsu (Calibrada)

In [9]:
viewer_otsu = napari.Viewer()
viewer_otsu.add_image(dapi_channel, name='DAPI Original', colormap='blue', scale=PHYSICAL_SCALE)
viewer_otsu.add_labels(otsu_mask, name='Máscara de Otsu', scale=PHYSICAL_SCALE)

<Labels layer 'Máscara de Otsu' at 0x740f70e15ca0>

## Paso 4: Segmentación de Núcleos con Cellpose
Ejecutamos Cellpose sobre el canal DAPI limpio y guardamos la máscara de etiquetas resultante.

In [14]:
# --- Importaciones adicionales para esta celda ---
from skimage.measure import regionprops_table
import pandas as pd

model = models.CellposeModel(gpu=True)
print("Ejecutando segmentación con Cellpose...")
cellpose_masks, _, _ = model.eval(
    dapi_channel_cleaned, 
    diameter=NUCLEUS_DIAMETER, 
    z_axis=0, 
    do_3D=True
)

# Guardamos la máscara de Cellpose
tifffile.imwrite(cellpose_mask_path, cellpose_masks.astype(np.uint16))
print(f"Máscara de Cellpose guardada en: {cellpose_mask_path}")

# --- ANÁLISIS CUANTITATIVO AÑADIDO ---
if cellpose_masks.max() > 0:
    # Calculamos propiedades: volumen (area en 3D) e intensidad media de DAPI
    props = regionprops_table(
        cellpose_masks,
        intensity_image=dapi_channel_cleaned,
        properties=('label', 'area', 'intensity_mean')
    )
    stats_df = pd.DataFrame(props).rename(columns={'area': 'volume_voxels'})
    
    print("\n--- Resultados de la Segmentación Inicial ---")
    print(f"Número total de núcleos encontrados: {len(stats_df)}")
    
    # Mostramos un resumen estadístico prolijo
    display(stats_df[['volume_voxels', 'intensity_mean']].describe().round(2))
else:
    print("\nNo se encontraron objetos en la segmentación inicial.")

Ejecutando segmentación con Cellpose...
Máscara de Cellpose guardada en: /home/daniel/Proyectos/astrocitos-3d-analysis/data/processed/Inmuno 26-07-23.lif - CTL 1-2 a/02_cellpose_mask.tif

--- Resultados de la Segmentación Inicial ---
Número total de núcleos encontrados: 363


,volume_voxels,intensity_mean
count,363.00,363.00
mean,1011.40,145.69
std,728.65,27.56
min,15.00,64.61
25%,332.00,126.70
50%,991.00,146.40
75%,1578.00,166.50
max,3418.00,208.63


### Visualización: Resultado de Cellpose (Calibrada)

In [15]:
viewer_cellpose = napari.Viewer()
viewer_cellpose.add_image(dapi_channel, name='DAPI Original', colormap='blue', scale=PHYSICAL_SCALE)
viewer_cellpose.add_labels(cellpose_masks, name='Resultado Cellpose (Crudo)', scale=PHYSICAL_SCALE)

<Labels layer 'Resultado Cellpose (Crudo)' at 0x740fb0064ad0>

## Paso 5: Filtrado por Co-localización GFAP

Iteramos sobre cada núcleo detectado y lo conservamos solo si la señal GFAP en su "anillo" peri-nuclear supera el umbral definido. Guardamos la máscara de los núcleos candidatos.

In [ ]:
DILATION_ITERATIONS = 40
GFAP_INTENSITY_THRESHOLD = 100

In [16]:
print("Filtrando núcleos por señal GFAP circundante...")
astrocyte_labels_candidate = []
# Re-calculamos props aquí por si las celdas se ejecutan fuera de orden
nuclei_props = regionprops(cellpose_masks, intensity_image=gfap_channel)

for nucleus in nuclei_props:
    nucleus_mask = (cellpose_masks == nucleus.label)
    dilated_mask = binary_dilation(nucleus_mask, iterations=DILATION_ITERATIONS)
    shell_mask = dilated_mask & ~nucleus_mask
    
    if np.any(shell_mask):
        shell_gfap_intensity = gfap_channel[shell_mask].mean()
        if shell_gfap_intensity > GFAP_INTENSITY_THRESHOLD:
            astrocyte_labels_candidate.append(nucleus.label)

gfap_filtered_mask = np.where(np.isin(cellpose_masks, astrocyte_labels_candidate), cellpose_masks, 0)

# Guardamos la máscara filtrada por GFAP
tifffile.imwrite(gfap_filtered_mask_path, gfap_filtered_mask.astype(np.uint16))
print(f"Máscara filtrada por GFAP guardada en: {gfap_filtered_mask_path}")

# --- FEEDBACK CUANTITATIVO AÑADIDO ---
print(f"\nProceso completado. Quedan {len(astrocyte_labels_candidate)} candidatos a astrocitos después del filtro GFAP.")

Filtrando núcleos por señal GFAP circundante...
Máscara filtrada por GFAP guardada en: /home/daniel/Proyectos/astrocitos-3d-analysis/data/processed/Inmuno 26-07-23.lif - CTL 1-2 a/03_gfap_filtered_mask.tif

Proceso completado. Quedan 0 candidatos a astrocitos después del filtro GFAP.


### Visualización: Candidatos a Astrocitos (Calibrada)

In [ ]:
viewer_gfap = napari.Viewer()
viewer_gfap.add_image(gfap_channel, name='GFAP', colormap='green', scale=PHYSICAL_SCALE)
viewer_gfap.add_labels(gfap_filtered_mask, name='Candidatos a Astrocitos', scale=PHYSICAL_SCALE)

Filtrando núcleos por señal GFAP circundante...
Máscara filtrada por GFAP guardada en: /home/daniel/Proyectos/astrocitos-3d-analysis/data/processed/Inmuno 26-07-23.lif - CTL 1-2 a/03_gfap_filtered_mask.tif


## Paso 6: Post-procesamiento por Tamaño Físico
Aplicamos el filtro final de limpieza, eliminando objetos con un volumen físico (µm³) menor al umbral definido.

In [ ]:
print("Aplicando filtro final de tamaño...")
final_props = regionprops(gfap_filtered_mask)
# Usamos el umbral en vóxeles calculado en la celda de parámetros
final_astrocyte_labels = [prop.label for prop in final_props if prop.area >= MIN_VOLUME_VOXELS]

final_mask = np.where(np.isin(gfap_filtered_mask, final_astrocyte_labels), gfap_filtered_mask, 0)

tifffile.imwrite(final_mask_path, final_mask.astype(np.uint16))
print(f"Máscara final guardada en: {final_mask_path}")

<Labels layer 'Candidatos a Astrocitos' at 0x7e99a416e450>

### Visualización: Resultado Final (Calibrada)
Visualizamos la imagen original multi-canal junto a la máscara final de astrocitos, todo con la escala física correcta.

In [ ]:
viewer_final = napari.Viewer()
viewer_final.add_image(
    image_stack, 
    channel_axis=1, 
    name=["DAPI", "GFAP", "Microglia"], 
    colormap=["blue", "green", "red"],
    scale=PHYSICAL_SCALE
)
viewer_final.add_labels(
    final_mask, 
    name='Astrocitos Finales',
    scale=PHYSICAL_SCALE
)

Aplicando filtro final de tamaño...
Máscara final guardada en: /home/daniel/Proyectos/astrocitos-3d-analysis/data/processed/Inmuno 26-07-23.lif - CTL 1-2 a/04_final_astrocytes_mask.tif
